What does this notebook do:

    - loads the charge distribution data.
    - computes the net heating per grain, full calculation.
    - takes the GTn parameter and computes the parametric net heating.
    - compares both distributions and gives the percentiles.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import pickle
import numpy as np
from   matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
import time

import compute_charge_dist as fz
import PeHeat_Functions as peh
import parametric_fz as parfz
import parametric_PeHeat as parPeH

Loading the library to compute the charge distribution of dust grains.


In [73]:
def index_fz(npoints, zminmax, fdist):

    ffz = []
    cum = 0
    for index in range(npoints):
        zmin = zminmax[index*2]
        zmax = zminmax[index*2+1]
        znum = int(abs(zmin) + zmax + 1)
        
        #print(zmin, zmax)
        #print(znum)
        
        ffz.append(0)
        ffz[-1] = []
        for j in range(znum):
            fhere = fdist[cum+j]
            ffz[-1].append(fhere)
            #print("filling the array", j)
        #print(ffz[-1])
        cum += znum
    
    return ffz

In [72]:
grain_type = "silicate"
#grain_type = "carbonaceous"

########################################################################################
pcent      = 1
grain_size = 3

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist3 = pickle.load(pkl_file)
pkl_file.close

Qabs3 = fz.get_QabsTable(grain_type, grain_size)

########################################################################################
pcent      = 1
grain_size = 5

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist5 = pickle.load(pkl_file)
pkl_file.close

Qabs5 = fz.get_QabsTable(grain_type, grain_size)

########################################################################################
pcent      = 1
grain_size = 10

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist10 = pickle.load(pkl_file)
pkl_file.close

Qabs10 = fz.get_QabsTable(grain_type, grain_size)

########################################################################################
pcent      = 1
grain_size = 50

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist50 = pickle.load(pkl_file)
pkl_file.close

Qabs50 = fz.get_QabsTable(grain_type, grain_size)

########################################################################################
pcent      = 1
grain_size = 100

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist100 = pickle.load(pkl_file)
pkl_file.close

Qabs100 = fz.get_QabsTable(grain_type, grain_size)

########################################################################################
pcent      = 0.1
grain_size = 500

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist500 = pickle.load(pkl_file)
pkl_file.close

Qabs500 = fz.get_QabsTable(grain_type, grain_size)

########################################################################################
pcent      = 0.1
grain_size = 1000

save_dir = "/home/jcibanezm/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist1000 = pickle.load(pkl_file)
pkl_file.close

Qabs1000 = fz.get_QabsTable(grain_type, grain_size)

I'm looking for a grain size smaller than 10 Angstrom. Outside the tabulated data!!!
Using the absorption efficiency for a 10 AA grain
I'm looking for a grain size smaller than 10 Angstrom. Outside the tabulated data!!!
Using the absorption efficiency for a 10 AA grain


In [74]:
fdist3 = index_fz(len(cdist3["nH"]), cdist3["zminmax"], cdist3["fdist"])
fdist5 = index_fz(len(cdist5["nH"]), cdist5["zminmax"], cdist5["fdist"])
fdist10 = index_fz(len(cdist10["nH"]), cdist10["zminmax"], cdist10["fdist"])

In [86]:
def Full_netHeating(grain_size, grain_type, nH, Gtot, temp, xe, zminmax, fdist, Qabs):
    """
    Given the charge distribution, and some ISM parameters, compute the net heating by a single dust grain.
    """
    Zmin_sil, Zmax_sil = fz.get_Zmin(grain_size, grain_type), fz.get_Zmax(grain_size, grain_type)
    Znum_sil = int(Zmax_sil + abs(Zmin_sil) +1)
    ZZ_sil = np.linspace(Zmin_sil, Zmax_sil, num=Znum_sil)

    Gamma_dotdot_Z_sil = np.zeros_like(ZZ_sil, dtype=np.float)

    for i in range(Znum_sil):
        Gamma_dotdot_Z_sil[i] = peh.get_Gamma_pe_dotdot(grain_size, ZZ_sil[i], grain_type, 0.0, Qabs, G0=1.7)

    Gamma_a = np.zeros_like(nH, dtype=np.float)
    Cooling = np.zeros_like(nH, dtype=np.float)
    cum = 0

    #for index in range(10):
    for index in range(len(Gamma_a)):

        zmin = zminmax[index*2]
        zmax = zminmax[index*2+1]
        znum = int(abs(zmin) + zmax + 1)

        Gamma_dotdot_Z_sil_scaled = Gamma_dotdot_Z_sil*(Gtot[index]/1.7)

        ZZfz = np.arange(zmin, zmax+1, 1)
        ffz  = fdist[index]
        #print(ZZfz)
        #print(ffz)

        Gamma_a[index]= peh.Gamma_per_grain(ZZ_sil, Gamma_dotdot_Z_sil_scaled, ZZfz, ffz)
        
        Cooling[index] = peh.Cool_per_Grain(grain_size, grain_type, ZZfz, ffz, nH[index], xe[index], temp[index])

    netHeat = Gamma_a - Cooling
    
    return netHeat

In [78]:
nH3, Gtot3, temp3, xe3, zminmax3, fdist3 = cdist3["nH"], cdist3["G"]+cdist3["G_CR"], cdist3["temp"],cdist3["new_xe"], cdist3["zminmax"], fdist3
nH5, Gtot5, temp5, xe5, zminmax5, fdist5 = cdist5["nH"], cdist5["G"]+cdist5["G_CR"], cdist5["temp"],cdist5["new_xe"], cdist5["zminmax"], fdist5
nH10, Gtot10, temp10, xe10, zminmax10, fdist10 = cdist10["nH"], cdist10["G"]+cdist10["G_CR"], cdist10["temp"],cdist10["new_xe"], cdist10["zminmax"], fdist10
nH50, Gtot50, temp50, xe50, zminmax50, fdist50 = cdist50["nH"], cdist50["G"]+cdist50["G_CR"], cdist50["temp"],cdist50["new_xe"], cdist50["zminmax"], cdist50["fdist"]
nH100, Gtot100, temp100, xe100, zminmax100, fdist100 = cdist100["nH"], cdist100["G"]+cdist100["G_CR"], cdist100["temp"],cdist100["new_xe"], cdist100["zminmax"], cdist100["fdist"]
nH500, Gtot500, temp500, xe500, zminmax500, fdist500 = cdist500["nH"], cdist500["G"]+cdist500["G_CR"], cdist500["temp"],cdist500["new_xe"], cdist500["zminmax"], cdist500["fdist"]
nH1000, Gtot1000, temp1000, xe1000, zminmax1000, fdist1000 = cdist1000["nH"], cdist1000["G"]+cdist1000["G_CR"], cdist1000["temp"],cdist1000["new_xe"], cdist1000["zminmax"], cdist1000["fdist"]

In [95]:
netHeat3 = Full_netHeating(3.5, grain_type, nH3, Gtot3, temp3, xe3, zminmax3, fdist3, Qabs3)
netHeat5 = Full_netHeating(5., grain_type, nH5, Gtot5, temp5, xe5, zminmax5, fdist5, Qabs5)
netHeat10 = Full_netHeating(10., grain_type, nH10, Gtot10, temp10, xe10, zminmax10, fdist10, Qabs10)
netHeat50 = Full_netHeating(50., grain_type, nH50, Gtot50, temp50, xe50, zminmax50, fdist50, Qabs50)
netHeat100 = Full_netHeating(100., grain_type, nH100, Gtot100, temp100, xe100, zminmax100, fdist100, Qabs100)
netHeat500 = Full_netHeating(500., grain_type, nH500, Gtot500, temp500, xe500, zminmax500, fdist500, Qabs500)
netHeat1000 = Full_netHeating(1000., grain_type, nH1000, Gtot1000, temp1000, xe1000, zminmax1000, fdist1000, Qabs1000)